In [1]:
from zipfile import ZipFile
from pathlib import Path

DSM_PATH = r"C:\Users\lilia\Downloads\SURVEY_LIDAR_Composite_ASC_DSM.zip"
with ZipFile(DSM_PATH, 'r') as zipObj:
    listOfFileNames = zipObj.namelist()

listOfFileNames = [Path(f).stem.split('_')[0].upper() for f in listOfFileNames]

with open('DSM_grid.txt', 'w') as f:
    for line in listOfFileNames:
        f.write(f"{line}\n")

In [2]:
# Load txt file with OSMap tiles
with open('osmapFileName.txt', 'r') as f:
    OS_files = f.read()
    OS_files = OS_files.split("\n")[:-1]

# Convert tile name to cardinal direction
coord_mapping = {
    (5,0): 'SE',
    (0,0): 'SW',
    (0,5): 'NW',
    (5,5): 'NE'
}

OS_tile_mapping = {}
for f in OS_files:
    tile_coord = (int(f[3]), int(f[-1]))
    tile_name = f[:3] + f[-2] + coord_mapping[tile_coord]
    
    OS_tile_mapping[tile_name] = f

In [3]:
# List out available DSM tiles for each OSMap tile
OS_files_dict = {}

for f in listOfFileNames:
    if int(f[3]) <5:
        tile_coord = f[:3] + '0'
    else:
        tile_coord = f[:3] + '5'

    if int(f[5]) < 5:
        tile_coord += f[4] + '0'
    else:
        tile_coord += f[4] + '5'

    OS_files_dict[f] = tile_coord

In [4]:
import pickle

with open('os_mapping.pkl', 'wb') as f:
    pickle.dump(OS_files_dict, f)

In [66]:
# Each 5kmx5km OSMap tile should have 25 1kmx1km DSM tiles
missing_tiles = {}
num_missing = 0
for k, v in OS_files_dict.items():
    missing_tiles[k] = 25-len(v)
    num_missing += len(v)

print(f"Total tiles: {25*len(OS_files_dict)}")
print(f"Number of missing tiles: {num_missing}")

Total tiles: 1450
Number of missing tiles: 946


In [63]:
# Save name of available tiles
with open('osmap_tiles.txt', 'w') as f:
    f.write('tile\tavailable\tnum_missing\n')
    for k,v in missing_tiles.items():
        f.write(f"{k}\t{OS_files_dict[k]}\t{v}\n")